# Подготовка к прогнозированию


In [1]:
import os
import pandas as pd 
import numpy as np

import matplotlib.pyplot as plt 
import seaborn as sns  
from IPython.display import Markdown


## Загрузка данных 

In [2]:
data_path = "C:\\Users\\Natalia\\Desktop\\Lenta_YM\\sp_sales_task\\"

if os.path.exists(f"{data_path}sales_df_train.csv"):
    top_50_rub_data = pd.read_csv(f"{data_path}top_50_rub_data.csv")
    sales_submission = pd.read_csv(f"{data_path}sales_submission.csv")
    holidays = pd.read_csv(f"{data_path}holidays.csv")
elif os.path.exists("top_50_rub_data.csv"):
    top_50_rub_data = pd.read_csv("top_50_rub_data.csv")
    sales_submission = pd.read_csv("sales_submission.csv") 
    holidays = pd.read_csv("holidays.csv")
else:
    print("Файлы не найдены. Проверьте путь к файлам.")

## Pipeline прогнозирования 

### Добаление признаков из исходного датасета в тестовую выборку  
В тестовую выборку необходимо добавить признки из исходного датасета.  
Ориентируемся на значения столбцов, содержащих названия товаров и магазинов, чтобы правильно заполнить строки тстовой выборки.

Преобразуем датафрейм, для которого нам необходимо заполнить целевые признаки, используя предобученную модель. 

**Добавление цены в качестве признака**

<img src="https://upload.wikimedia.org/wikipedia/commons/b/ba/Warning_sign_4.0.png" align=left width=44, heigth=33>

**Отметим, что выручка за товар pr_sales_in_rub, которую мы имеем в исходном датасете находится не в прямой зависимости от себестоимости. На нее влияют промо-акции, которые мы спрогнозировать не можем, также на нее влияет поток покупателей с определенными льготами (например, карта Ленты, социальные карты и др. льготы) и пр.**


По паре товар-магазин нужно добавить цену в качестве признака. 

Это можно сделать несколькими способами:
1. спрогнозировать с помощью Prophet или Arima или другой модели. Но на это требуется больше времени, мы не успеваем в рамках хакатона реализовать задумку. 
2. Заполнить данные по цене средним значением цены за последние 14 дней. Это будет честным (не будет считаться подглядыванием), на мой взгляд, поскольку цена на продукты в течение двух недель не сильно меняется (если это не кризис или еще что-то). Например, если я покупаю хлеб, за 50 рублей, то и через две недели он будет скорее всего 50руб. 
3. Можно заложить инфляцию (официальную) на продукт, но это уже слишком усложняет манипуляции с ценой, но едва ли близко к правде. 
4. Заполнить срезней ценой по каждой паре товар-магазин по всему датасету. 

Будем заполнять данные по цене средней ценой по каждой паре товар-магазин. 

In [3]:
# top_50_rub_data.info()

In [4]:
holidays['date'] = pd.to_datetime(holidays['date'])
# holidays.head(5)

In [5]:
# holidays.info()

In [6]:
sales_submission['date'] = pd.to_datetime(sales_submission['date'])
sales_submission = sales_submission.drop(['target'], axis=1)
# sales_submission.info()

In [7]:
sales_submission = sales_submission.merge(holidays, on='date', how='left')
sales_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43694 entries, 0 to 43693
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   st_id      43694 non-null  object        
 1   pr_sku_id  43694 non-null  object        
 2   date       43694 non-null  datetime64[ns]
 3   holiday    43694 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 1.3+ MB


Для обучения модели и для ее тестирования нужны датасеты, содержащие одинаковые колонки признаков. То есть к датасету sales_submission необходимо добавить недостающие колонки из обучающей выборки (например, группа товаров или локация магазина), а также спрогнозировать те признаки и подать в файл, которые нелхя скопировтаь (например, цена). 

In [8]:
top_50_rub_data_modifed = top_50_rub_data.copy()
top_50_rub_data_modifed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59786 entries, 0 to 59785
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   date               59786 non-null  object 
 1   st_id              59786 non-null  object 
 2   pr_sku_id          59786 non-null  object 
 3   pr_sales_in_units  59786 non-null  float64
 4   pr_sales_in_rub    59786 non-null  float64
 5   holiday            59786 non-null  int64  
 6   pr_group_id        59786 non-null  object 
 7   pr_cat_id          59786 non-null  object 
 8   pr_subcat_id       59786 non-null  object 
 9   pr_uom_id          59786 non-null  int64  
 10  st_city_id         59786 non-null  object 
 11  st_division_code   59786 non-null  object 
 12  st_type_format_id  59786 non-null  int64  
 13  st_type_loc_id     59786 non-null  int64  
 14  st_type_size_id    59786 non-null  int64  
 15  product_price      59786 non-null  float64
 16  average_prices     597

In [10]:
top_50_rub_data_modifed = top_50_rub_data_modifed.drop([
    'pr_sales_in_rub', 'pr_cat_id', 'st_division_code', 
    'pr_group_id', 'st_city_id', 'pr_subcat_id',
    'pr_sales_in_units', 'product_price', 'holiday', 'date'], axis=1)

In [11]:
top_50_rub_data_modifed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59786 entries, 0 to 59785
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   st_id              59786 non-null  object 
 1   pr_sku_id          59786 non-null  object 
 2   pr_uom_id          59786 non-null  int64  
 3   st_type_format_id  59786 non-null  int64  
 4   st_type_loc_id     59786 non-null  int64  
 5   st_type_size_id    59786 non-null  int64  
 6   average_prices     59786 non-null  float64
dtypes: float64(1), int64(4), object(2)
memory usage: 3.2+ MB


Нам необходимо оставть только уникальные сочетания для пар товар-магазин. Найдем и удалим явные дубликаты

In [12]:
print('Количество явных дубликатов:', top_50_rub_data_modifed.duplicated().sum())

Количество явных дубликатов: 59500


In [13]:
top_50_rub_data_modifed = top_50_rub_data_modifed.drop_duplicates()
top_50_rub_data_modifed.info() 

<class 'pandas.core.frame.DataFrame'>
Index: 286 entries, 0 to 45634
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   st_id              286 non-null    object 
 1   pr_sku_id          286 non-null    object 
 2   pr_uom_id          286 non-null    int64  
 3   st_type_format_id  286 non-null    int64  
 4   st_type_loc_id     286 non-null    int64  
 5   st_type_size_id    286 non-null    int64  
 6   average_prices     286 non-null    float64
dtypes: float64(1), int64(4), object(2)
memory usage: 17.9+ KB


In [14]:
top_50_rub_data_modifed.head() 

,st_id,pr_sku_id,pr_uom_id,st_type_format_id,st_type_loc_id,st_type_size_id,average_prices
0,fa7cdfad1a5aaf8370ebeda47a1ff1c3,c4a665596d4f67cecb7542c9fad407ee,1,1,1,12,123.216649
1,c81e728d9d4c2f636f067f89cc14862c,dce1f234d6424aa61f8e7ce0baffd9af,1,1,1,8,135.978656
2,f7e6c85504ce6e82442c770f7c8606f0,58ebafabd92e2e3a80d86b7bb7e88eda,1,1,1,12,192.968946
3,c81e728d9d4c2f636f067f89cc14862c,777d21d980f82aa8b4f26f4a4d3dda1d,1,1,1,8,290.611062
4,c81e728d9d4c2f636f067f89cc14862c,0a2090e24b6ae62b0b0fcaa67a72b5a0,1,1,1,8,135.325400


**Выполним обьединение полученных датафреймов** 

Выполним внутреннее объединение (inner join) между subm_example и top_50_rub_data по столбцам 'st_id' и 'pr_sku_id', и результат будет содержать только строки, где значения в этих двух столбцах совпадают.

Так мы избавимся от товаров, которые не вощли в нашу выорку топ-50 товаров и добавим информацию по товарам и магазинам, которую использовали в трейне. 
То есть, например, информация о локации магазина не будет подглядыванием, также как и то, что багет относится к категории хлеб. Эту информацию мы и добавляем. 

In [15]:
submission_modified = sales_submission.merge(top_50_rub_data_modifed, on=['st_id', 'pr_sku_id'], how='inner')
# submission_modified.info()

In [16]:
cat_features = ['st_id', 'pr_sku_id']
for col in cat_features:
    submission_modified[col] = submission_modified[col].astype('category')

In [17]:
# submission_modified

In [18]:
submission_modified = submission_modified.set_index('date')
submission_modified.sort_index(inplace=True)

In [19]:
submission_modified = submission_modified.rename(columns={'average_prices': 'product_price'})

In [20]:
submission_modified.head()

,st_id,pr_sku_id,holiday,pr_uom_id,st_type_format_id,st_type_loc_id,st_type_size_id,product_price
date,,,,,,,,
2023-07-19,42a0e188f5033bc65bf8d78622277c4e,58ebafabd92e2e3a80d86b7bb7e88eda,0,1,1,1,12,192.968946
2023-07-19,c81e728d9d4c2f636f067f89cc14862c,cf91eca9321c9c272fb4e7cf4bcc6a43,0,17,1,1,8,36.348641
2023-07-19,42a0e188f5033bc65bf8d78622277c4e,5d560ea997068115892d2f0bd7cf91c3,0,1,1,1,12,165.998658
2023-07-19,c81e728d9d4c2f636f067f89cc14862c,0a68dd43c227b0e66d52665d6c3ca8ba,0,1,1,1,8,142.667930
2023-07-19,fa7cdfad1a5aaf8370ebeda47a1ff1c3,0a2090e24b6ae62b0b0fcaa67a72b5a0,0,1,1,1,12,135.325400


In [21]:
submission_modified.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3836 entries, 2023-07-19 to 2023-08-01
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   st_id              3836 non-null   category
 1   pr_sku_id          3836 non-null   category
 2   holiday            3836 non-null   int64   
 3   pr_uom_id          3836 non-null   int64   
 4   st_type_format_id  3836 non-null   int64   
 5   st_type_loc_id     3836 non-null   int64   
 6   st_type_size_id    3836 non-null   int64   
 7   product_price      3836 non-null   float64 
dtypes: category(2), float64(1), int64(5)
memory usage: 220.1 KB


In [22]:
submission_modified.to_csv('submission_modified_2.csv', index=True)

**ВЫВОД:** 

- Подготовлен submission_modified файл для получения sales_submission прогноза. 
- В ходе подготовки к sales_submission файлу добавлены дополнительне колонки прзнаков, которые использовались при обучении модели - выходные и праздничные дни, локация и размер магазина, маркер товара (шт или г), формат магазина. В качестве цены за единицу товара была добалена средняя цена, рассчитанная за весь период для пар товар-магазин. 
- Также из изначального sales_submission файла исключены товары, для которых мы не делали обучение модели. Остались только те товары, которые входили в топ-50 отобранных по выручке. 
- Полученный файл submission_modified подается на вход обученной модели, чтобы получить предсказания на следующие 14 дней по топ-50 товарам